In [50]:
import tmdbsimple as tmdb
import pandas as pd
import numpy as np
import seaborn as sns
import urllib.request
import requests
import configparser

In [52]:
config = configparser.ConfigParser()
config.read('API_KEYS.ini')
config.sections()

API_KEY = config['tmdb.com']['API_KEY']

tmdb.API_KEY = API_KEY

In [53]:
genome_tags_ids = pd.read_csv('data/genome-tags.csv')
movies = pd.read_csv('data/movies.csv')
genome_tags = pd.read_csv('data/genome-scores.csv')
user_tags = pd.read_csv('data/tags.csv')
links = pd.read_csv('data/links.csv')

In [54]:
#45843 movies
movies.shape

(45843, 3)

In [55]:
movies = movies.merge(links, left_on = 'movieId', right_on = 'movieId', how = 'left')
movies.dropna(subset=['tmdbId'], inplace=True)
movies['tmdbId'] = movies['tmdbId'].astype(int)
movies.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357
4,5,Father of the Bride Part II (1995),Comedy,113041,11862


In [56]:
genome_tags = genome_tags.merge(genome_tags_ids, left_on = 'tagId', right_on = 'tagId', how = 'left')
genome_tags = genome_tags.merge(movies[['title', 'movieId', 'tmdbId']], left_on = 'movieId', right_on = 'movieId', how = 'left')
genome_tags = genome_tags[genome_tags['tmdbId'].notnull()]
genome_tags['tmdbId'] = genome_tags['tmdbId'].astype(int)
genome_tags.head()

,movieId,tagId,relevance,tag,title,tmdbId
0,1,1,0.02475,007,Toy Story (1995),862
1,1,2,0.02475,007 (series),Toy Story (1995),862
2,1,3,0.04900,18th century,Toy Story (1995),862
3,1,4,0.07750,1920s,Toy Story (1995),862
4,1,5,0.12450,1930s,Toy Story (1995),862


In [57]:
#Only 10982 movies with genome tags
len(genome_tags['movieId'].unique())

10982

In [58]:
x = genome_tags.pivot(index='movieId', columns='tag', values='relevance')
#x = x.reset_index()
x.columns = map(str.lower, x.columns)
x.columns = x.columns.str.replace(' ', '_')

In [59]:
x.head()

,007,007_(series),18th_century,1920s,1930s,1950s,1960s,1970s,1980s,19th_century,...,world_politics,world_war_i,world_war_ii,writer's_life,writers,writing,wuxia,wwii,zombie,zombies
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02475,0.02475,0.04900,0.07750,0.12450,0.23875,0.06575,0.28575,0.25400,0.02625,...,0.03125,0.02050,0.04300,0.03375,0.12375,0.04150,0.02125,0.03600,0.10425,0.02750
2,0.03750,0.04100,0.03675,0.04750,0.10000,0.05950,0.05125,0.09600,0.08875,0.01875,...,0.03425,0.01825,0.01650,0.02325,0.13525,0.02450,0.01825,0.01325,0.08550,0.01925
3,0.03800,0.04825,0.02650,0.07300,0.05475,0.06025,0.05750,0.15675,0.03900,0.02900,...,0.03700,0.02450,0.02650,0.03125,0.15700,0.03650,0.01750,0.01725,0.10525,0.02075
4,0.03125,0.03400,0.03375,0.02875,0.06175,0.03125,0.02250,0.07250,0.04400,0.02775,...,0.04650,0.03200,0.02150,0.03850,0.19150,0.06050,0.01725,0.01325,0.05650,0.01150
5,0.04100,0.05125,0.05550,0.03300,0.07475,0.11100,0.02900,0.07150,0.02075,0.02875,...,0.03575,0.02900,0.01875,0.02125,0.14300,0.02375,0.01500,0.01325,0.12125,0.02125


In [60]:
u, s, vh = np.linalg.svd(x.loc[:1000, :], full_matrices=True)

In [61]:
y = vh[0,:]
sorted(range(len(y)), key=lambda k: y[k]*-1)

[118,
 784,
 656,
 14,
 572,
 488,
 218,
 116,
 346,
 592,
 632,
 975,
 783,
 339,
 491,
 560,
 1001,
 103,
 774,
 105,
 541,
 1059,
 391,
 635,
 963,
 13,
 70,
 558,
 737,
 1079,
 697,
 664,
 679,
 672,
 858,
 102,
 1068,
 39,
 1029,
 320,
 698,
 964,
 1078,
 540,
 677,
 785,
 16,
 576,
 938,
 1076,
 289,
 1098,
 934,
 294,
 631,
 224,
 399,
 763,
 37,
 939,
 956,
 945,
 206,
 487,
 46,
 809,
 705,
 937,
 319,
 479,
 992,
 160,
 825,
 73,
 173,
 735,
 589,
 533,
 40,
 209,
 398,
 1093,
 1037,
 872,
 500,
 1055,
 403,
 91,
 186,
 1008,
 341,
 855,
 590,
 771,
 493,
 1105,
 953,
 692,
 911,
 475,
 691,
 208,
 694,
 87,
 1066,
 570,
 1075,
 1127,
 212,
 721,
 974,
 130,
 1074,
 961,
 183,
 113,
 739,
 969,
 146,
 514,
 847,
 141,
 38,
 122,
 913,
 658,
 64,
 199,
 1038,
 926,
 723,
 923,
 865,
 88,
 236,
 306,
 204,
 135,
 559,
 23,
 485,
 634,
 1035,
 666,
 253,
 722,
 661,
 316,
 158,
 345,
 1018,
 309,
 55,
 272,
 649,
 115,
 455,
 216,
 90,
 76,
 814,
 1047,
 210,
 905,
 643,
 753,
 

In [62]:
y.min()

-0.14645519886574307

In [63]:
print(x.columns[118])
print(x.columns[784])
print(x.columns[656])
print(x.columns[14])
print(x.columns[572])
print(x.columns[488])


beatles
pixar
miyazaki
aardman
james_bond
hannibal_lecter


In [66]:
#get list of top correlated variables
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(x, 100))

Top Absolute Correlations
vampire                                   vampires                               0.985200
nazi                                      nazis                                  0.972703
super_hero                                superhero                              0.962798
post_apocalyptic                          post-apocalyptic                       0.960597
sci_fi                                    scifi                                  0.960092
miyazaki                                  studio_ghibli                          0.958161
world_war_ii                              wwii                                   0.956744
feel_good_movie                           feel-good                              0.956563
alien                                     aliens                                 0.952959
animated                                  animation                              0.952823
super_hero                                superheroes                     

In [ ]:
# Create correlation of continuous columns# Create 
corr = x.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin = -1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .75})

In [24]:
movie = tmdb.Movies(8844)

In [25]:
movie.info()

{'adult': False,
 'backdrop_path': '/7k4zEgUZbzMHawDaMc9yIkmY1qR.jpg',
 'belongs_to_collection': {'id': 495527,
  'name': 'Jumanji Collection',
  'poster_path': '/1dS95mhJHJ6OZHXRnoix5RRxu3X.jpg',
  'backdrop_path': '/dsaUpJoxSqYJh96s5Z9hHzoRKQ3.jpg'},
 'budget': 65000000,
 'genres': [{'id': 12, 'name': 'Adventure'},
  {'id': 14, 'name': 'Fantasy'},
  {'id': 10751, 'name': 'Family'}],
 'homepage': None,
 'id': 8844,
 'imdb_id': 'tt0113497',
 'original_language': 'en',
 'original_title': 'Jumanji',
 'overview': "When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",
 'popularity': 5.098,
 'poster_path': '/vgpXmVaVyUL7GGiDeiK1mKEKzcX.jpg',
 'productio

In [21]:
pd.DataFrame([response], columns=response.keys())

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/dji4Fm0gCDVb9DQQMRvAI8YNnTz.jpg,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,1995-10-30,373554033,81,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.9,8321


In [10]:
movie.info()['poster_path']

'/rhIRbceoE9lR4veEXuwCC2wARtG.jpg'

In [14]:
import os
os.chdir('data/images')

In [15]:
x, y = urllib.request.urlretrieve("https://image.tmdb.org/t/p/w400/rhIRbceoE9lR4veEXuwCC2wARtG.jpg", "toy_story.jpg")

In [16]:
print(x)

toy_story.jpg


In [39]:
result = requests.get("https://image.tmdb.org/t/p/w400/rhIRbceoE9lR4veEXuwCC2wARtG.jpg", stream=True)
if result.status_code == 200:
    image = result.raw.read()
    open('/Users/y060026/Springboard/projects/movie_poster_analysis/data/images/toy_story.jpg',"wb").write(image)